#### Gerekli Kütüphaneler 

In [1]:
import numpy as np
import pandas as pd

#### Veri Yükleme 

Veriseti kaynak: https://www.kaggle.com/suleymancan/turkishnews70000

In [2]:
df = pd.read_csv("turkish_news_70000.csv",index_col = "id")

# Veri setindeki ilk üç satır 
df.head(3)

,main_image,published,site,text,title,url
id,,,,,,
72337,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T13:33:00.000+02:00""","""diken.com.tr""","""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...","""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...","""http://www.diken.com.tr/yatirim-bankasi-dolar..."
72338,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T13:21:00.000+02:00""","""diken.com.tr""","""PİAR Araştırma: Adana ve Antalya’da ‘millet i...","""PİAR Araştırma: Adana ve Antalya’da ‘millet i...","""http://www.diken.com.tr/piar-arastirma-adana-..."
72339,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T12:20:00.000+02:00""","""diken.com.tr""","""Renaissance Capital: Merkez Bankası bu hafta ...","""Renaissance Capital: Merkez Bankası bu hafta ...","""http://www.diken.com.tr/renaissance-capital-m..."


In [3]:
# Veriseti farklı kaynaklardan elde edilen, 70000 haber içeriğinden oluşmaktadır.
# Örnek uygulama gösterdiğimizden dolayı, ilk 10000 haber içeriğini kullanacağız.
df = df.head(10000)

#### Veri Hazırlama & Temizleme

In [4]:
# LDA modelinde sadece "text" sütununu kullanmamız yeterli olacaktır.
haber_veriseti = df[["text"]]
haber_veriseti.head(3)

,text
id,
72337,"""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9..."
72338,"""PİAR Araştırma: Adana ve Antalya’da ‘millet i..."
72339,"""Renaissance Capital: Merkez Bankası bu hafta ..."


#### Başlıca Veri Temizleme İşlemleri
#### 1) Verisetindeki tüm harfleri küçük duruma getirmek
#### 2) Noktalama işaretlerini kaldırmak
#### 3) Etkisiz kelimeleri  (stopwords) kaldırmak. Örnek: ve, için, ama, çok, vb.

In [5]:
# Veri temizleme işlemi için, bir fonksiyon tanımlayalım ve 
# Bu fonksiyonu bütün veri üzerinde uygulayalım
# Öncesinde noktalama işaretleri ve storwords kümelerini tanımlayalım
import re
import string
import nltk
from nltk.corpus import stopwords

nok_isaretleri_kümesi = string.punctuation
# nltk kütüphanesinin etkisiz kelimeler kümesini kullanalım 
etkisiz_kelimeler_kümesi = stopwords.words('turkish') 
# stopwords kümesine biz de istediğimiz kelimeleri ekleyebiliriz
# Örnek:
etkisiz_kelimeler_kümesi.extend(["bir","kadar","sonra"])

def veri_temizleme(metin):
    # Metindeki tüm harfleri küçük duruma getirir.
    metin = metin.lower()
    # Verisetimizdeki yeni satır karekterleri, boşluk karekteriyle değiştirdik.  
    metin = metin.replace("\\n"," ")
    # Kesme işareti ve sonrasındaki karekterlerin kaldırılması
    metin = re.sub("’(\w+)", "", metin) 
    metin = re.sub("'(\w+)", "", metin) 
    metin = re.sub("[“,‘,’,”]", "", metin) 
    # Sayıların Kaldırılması
    metin = re.sub("[0-9]+", "", metin)
    # Noktalama işaretlerinin kaldırılması
    metin = "".join(list(map(lambda x:x if x not in nok_isaretleri_kümesi else " ", metin)))
    # Etkisiz kelimelerin bir kısmının kaldırılması
    metin = " ".join([i for i in metin.split() if i not in etkisiz_kelimeler_kümesi])
    # Metinde tek kalan harfleri de çıkartalım
    metin = " ".join([i for i in metin.split() if len(i) > 1])
    
    return metin

In [6]:
haber_veriseti["temizlenen_metin"] = haber_veriseti["text"].apply(lambda x:veri_temizleme(x))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Ham Metin - Temizlenmiş Metin Örneği

In [7]:
haber_veriseti.iloc[5].text

'"Şubat ayında ihracat yüzde 3.7 arttı, ithalat yüzde 18.7 azaldı 04/03/2019 12:20\\nTicaret Bakanı Ruhsar Pekcan şubat ayında ihracatın yüzde 3.7 artışla 14 milyar 312 milyon dolar, ithalatın yüzde 18.7 azalışla 16 milyar 161 milyon dolar olarak gerçekleştiğini açıkladı. Fotoğraf: Reuters\\nBakan Pekcan şunları söyledi: “ Eskiden kullandığımız Özel Ticaret Sistemine göre de şubat ayında ihracatımız yüzde 3,5 artışla 13 milyar 603 milyon dolar olarak gerçekleşmiştir. ÖTS’ye göre ithalatımız şubat ayında yüzde 16,6 düşüşle 15 milyar 793 milyon dolar seviyesinde gerçekleşmiştir. ” Reklam"'

In [8]:
haber_veriseti.iloc[5].temizlenen_metin

'şubat ayında ihracat yüzde arttı ithalat yüzde azaldı ticaret bakanı ruhsar pekcan şubat ayında ihracatın yüzde artışla milyar milyon dolar ithalatın yüzde azalışla milyar milyon dolar olarak gerçekleştiğini açıkladı fotoğraf reuters bakan pekcan şunları söyledi eskiden kullandığımız özel ticaret sistemine göre şubat ayında ihracatımız yüzde artışla milyar milyon dolar olarak gerçekleşmiştir öts göre ithalatımız şubat ayında yüzde düşüşle milyar milyon dolar seviyesinde gerçekleşmiştir reklam'

#### Metin Tokenize Etme İşlemi 

In [9]:
haber_veriseti["temizlenen_metin_token"] = haber_veriseti["temizlenen_metin"].apply(lambda x: x.split())

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
# Temizlenmiş ve tokenize edilmiş metinleri de haber_veriseti dataframe'ine ekledik
# İlk on satır örnek
haber_veriseti.head(10)

,text,temizlenen_metin,temizlenen_metin_token
id,,,
72337,"""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...",yatırım bankası dolar tl üçüncü çeyrekte görec...,"[yatırım, bankası, dolar, tl, üçüncü, çeyrekte..."
72338,"""PİAR Araştırma: Adana ve Antalya’da ‘millet i...",pi̇ar araştırma adana antalya millet ittifakı ...,"[pi̇ar, araştırma, adana, antalya, millet, itt..."
72339,"""Renaissance Capital: Merkez Bankası bu hafta ...",renaissance capital merkez bankası hafta baz p...,"[renaissance, capital, merkez, bankası, hafta,..."
72340,"""Ağrı’daki İYİ Partililerin istifa gerekçesi: ...",ağrı i̇yi̇ partililerin istifa gerekçesi akp s...,"[ağrı, i̇yi̇, partililerin, istifa, gerekçesi,..."
72341,"""Otomobil pazarı iki ayda yarı yarıya eridi 04...",otomobil pazarı iki ayda yarı yarıya eridi oto...,"[otomobil, pazarı, iki, ayda, yarı, yarıya, er..."
72342,"""Şubat ayında ihracat yüzde 3.7 arttı, ithalat...",şubat ayında ihracat yüzde arttı ithalat yüzde...,"[şubat, ayında, ihracat, yüzde, arttı, ithalat..."
72343,"""YSK Başkanı Güven'den 'seçim güvenliği' açıkl...",ysk başkanı güven güvenliği açıklaması şubat c...,"[ysk, başkanı, güven, güvenliği, açıklaması, ş..."
72344,"""Hapisten çıkmasın diye kızları intihara kalkı...",hapisten çıkmasın kızları intihara kalkışmıştı...,"[hapisten, çıkmasın, kızları, intihara, kalkış..."
72345,"""Yavaş’tan Özhaseki’ye: Tecrübeyse, Gökçek dah...",yavaş özhaseki tecrübeyse gökçek tecrübeliydi ...,"[yavaş, özhaseki, tecrübeyse, gökçek, tecrübel..."


####  LDA Model Eğitimi

Kaynak: https://radimrehurek.com/gensim/models/ldamodel.html

In [11]:
import gensim
import pyLDAvis.gensim # LDA Konu Modellemesi figürsel gösterimi için kullanılan kütüphane 

In [13]:
# Kelime Listesi - Dictionary Oluşturulması
tokenlastirilmis_metinler = haber_veriseti["temizlenen_metin_token"]
kelime_listesi = gensim.corpora.Dictionary(tokenlastirilmis_metinler)

#Kelime Listesi Filtreleme
kelime_listesi.filter_extremes(no_below=1, no_above=0.7)

In [14]:
# Terimlerin Vektörleştirilmesi -- Doküman-Terim Matrisinin Oluşturulması
dokuman_terim_matrisi = [kelime_listesi.doc2bow(terim) for terim in tokenlastirilmis_metinler]

In [ ]:
# LDA Model
lda_model = gensim.models.ldamodel.LdaModel(corpus = dokuman_terim_matrisi,
                                           id2word = kelime_listesi,
                                           num_topics = 15, 
                                           passes = 10)

#### Sonuçlar 

In [ ]:
# Oluşturulan soyut konular içerisinde en fazla bulunan terimler
konular = lda_model.print_topics(num_words=7) 

for konu in konular:
    print(konu)

In [ ]:
pyLDAvis.enable_notebook()
gorsel = pyLDAvis.gensim.prepare(lda_model, belge_terim_matrisi, sozluk, mds='mmds')
pyLDAvis.display(gorsel)

#### Tutarlılık (Coherence) Skoru ile En iyi Konu Sayısı Seçimi 

In [12]:
from gensim.models import CoherenceModel

konu_sayisi_aralik_listesi = range(9,30,3)

In [15]:
tutarlilik_skorlar_listesi = list()
konu_sayisi_listesi = list()

for konu_sayisi in konu_sayisi_aralik_listesi:
    lda_model = gensim.models.ldamodel.LdaModel(corpus = dokuman_terim_matrisi,
                                               id2word = kelime_listesi,
                                               num_topics = konu_sayisi, 
                                               passes = 10)
    
    tutarlilik_model_lda = CoherenceModel(model=lda_model, texts=tokenlastirilmis_metinler, 
                                          dictionary=kelime_listesi, coherence='c_v')
    gecici_tutarlilik_skoru_lda = tutarlilik_model_lda.get_coherence()
    tutarlilik_skorlar_listesi.append(gecici_tutarlilik_skoru_lda)
    konu_sayisi_listesi.append(konu_sayisi)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(konu_sayisi_listesi, tutarlilik_skorlar_listesi, "-"),
plt.xlabel("Konu Sayıları")
plt.ylabel("Tutarlılık Skorları")

plt.show()

In [47]:
# LDA Model
# num_topics parametresi, görselde en iyi çıkan tutarlılık skoruna göre belirlenebilir. 
# En yüksek tutarlılık sokurunu veren konu sayısı seçilebilir.

lda_model = gensim.models.ldamodel.LdaModel(corpus = dokuman_terim_matrisi,
                                           id2word = kelime_listesi,
                                           num_topics = 25, 
                                           passes = 10)

In [ ]:
# Oluşturulan soyut konular içerisinde en fazla bulunan terimler
konular = lda_model.print_topics(num_words=7) 
konular = sorted(konular, key = lambda x: x[0]) 
for konu in konular:
    print(konu)

In [25]:
pyLDAvis.enable_notebook()
gorsel = pyLDAvis.gensim.prepare(lda_model, dokuman_terim_matrisi, kelime_listesi, mds='mmds')
pyLDAvis.display(gorsel)